In [0]:
%sql
-- ============================================
-- BRONZE LAYER (Raw Data Ingestion)
-- ============================================

-- Create Bronze Table from original uploaded table
Create or replace table bronze_sales AS 
Select 
`Row ID` as Row_Id,`Order ID`as Order_Id,`Order Date` as Order_Date,`Ship Date` as Ship_Date,`Ship Mode` as Ship_Mode,`Customer ID` as Customer_Id,`Customer Name` as Customer_Name,`Segment` as Segment,`Country` as Country ,`City` as City,`State` as State,	`Postal Code` as Postal_Code,`Region` as Region,`Product ID` as Product_Id,`Category` as Category,`Sub-Category` as Sub_Category,`Product Name` as Product_Name,`Sales` as Sales
from sales_data;


In [0]:
%sql
---- Check record count
SELECT COUNT(*) AS total_records
FROM bronze_sales;



In [0]:
%sql
-- Check table format
DESCRIBE DETAIL bronze_sales;

In [0]:
%sql
-- ============================================
-- SILVER LAYER (Data Cleaning & Validation)
-- ============================================

CREATE OR REPLACE TABLE silver_sales AS
SELECT
    Ship_Date,
    Region,
    State,
    Category,
    Sub_Category,
    Product_Name,
    CAST(Sales AS DOUBLE) AS Sales
FROM bronze_sales
WHERE Sales IS NOT NULL;

In [0]:
%sql
-- Data Quality Checks
-- Null check
SELECT COUNT(*) AS null_sales
FROM bronze_sales
WHERE Sales IS NULL;


-- Negative sales check
SELECT *
FROM silver_sales
WHERE Sales < 0;

-- Duplicate check
SELECT Product_Name, COUNT(*) AS record_count
FROM bronze_sales
GROUP BY Product_Name
HAVING COUNT(*) > 1;

In [0]:
%sql
-- ============================================
-- GOLD LAYER (Business Aggregations)
-- ============================================
CREATE OR REPLACE TABLE gold_sales_summary AS
SELECT Region,
    Category,
    SUM(Sales) AS total_sales
FROM silver_sales
GROUP BY Region, Category;


-- View Gold Layer
SELECT * FROM gold_sales_summary;

In [0]:
%sql

-- ============================================
-- DELTA CONVERSION & OPTIMIZATION
-- ============================================

--To check the table format of the table
DESCRIBE DETAIL silver_sales;

-- Convert to Delta (if not already)
--CONVERT TO DELTA bronze_sales;

-- Optimize table
OPTIMIZE silver_sales;

-- Z-Order for performance
OPTIMIZE silver_sales
ZORDER BY (Region, Category);

-- View history (Time Travel feature)
DESCRIBE HISTORY silver_sales;

-- Example Time Travel Query(If converted)
SELECT * 
FROM silver_sales VERSION AS OF 1;

-- Example Time Travel Query( If not conveted)
SELECT * 
FROM silver_sales VERSION AS OF 0;

In [0]:
%sql

-- Monthly Sales Trend
SELECT 
    YEAR(TO_DATE(`ship date`, 'yyyy-MM-dd')) AS year,
    MONTH(TO_DATE(`ship date`, 'yyyy-MM-dd')) AS month,
    SUM(sales) AS monthly_sales
FROM sales_data
GROUP BY 
    YEAR(TO_DATE(`ship date`, 'yyyy-MM-dd')),
    MONTH(TO_DATE(`ship date`, 'yyyy-MM-dd'))
ORDER BY year, month;

-- Top 5 Products by Sales
SELECT 
    Product_Name,
    SUM(Sales) AS total_sales
FROM silver_sales
GROUP BY Product_Name
ORDER BY total_sales DESC
LIMIT 5;

-- Running Total Sales by Region
SELECT
    Region,
    Ship_Date,
    SUM(Sales) OVER (
        PARTITION BY Region
        ORDER BY Ship_Date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS running_sales
FROM silver_sales;
